# ⚡ 微电网数字孪生系统 | Microgrid Digital Twin System

---

## 🎯 系统特性

- **☀️ 光伏发电系统** - 3D太阳能电池板阵列可视化
- **🔋 储能系统** - 电池组SOC实时监控
- **🏠 负荷管理** - 多类型建筑负荷模拟
- **⚡ 电网交互** - 购售电能量流动
- **📊 实时数据** - 功率曲线与能量统计

---

**运行说明**: 按顺序执行下面的代码单元格即可启动系统

## 📦 第1步: 安装依赖库

In [ ]:
# 安装必要的库
!pip install plotly ipywidgets -q

# 启用Colab的plotly支持
import plotly.io as pio
pio.renderers.default = 'colab'

print("✅ 依赖库安装完成!")

## 🔧 第2步: 导入核心模块

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
from datetime import datetime, timedelta
import random

print("✅ 模块导入成功!")

## 🎨 第3步: 定义颜色主题和3D模型

In [ ]:
# ============================================================================
# 颜色主题配置
# ============================================================================
COLORS = {
    'solar': '#FFD700',      # 金黄色 - 太阳能
    'battery': '#00CED1',    # 青色 - 储能
    'load': '#FF6347',       # 番茄红 - 负荷
    'grid': '#9370DB',       # 紫色 - 电网
    'flow_positive': '#00FF00',  # 绿色 - 正向能量流
    'flow_negative': '#FF4500',  # 橙红色 - 反向能量流
    'background': '#1a1a2e',     # 深蓝黑背景
    'panel': '#16213e',          # 面板背景
    'text': '#eee'               # 文字颜色
}

print("✅ 颜色主题已配置!")

In [ ]:
# ============================================================================
# 3D 模型生成函数
# ============================================================================

def create_solar_panel_3d(x_offset=0, y_offset=0, z_offset=0, scale=1.0, tilt_angle=30):
    """
    创建3D太阳能电池板模型
    """
    width = 2.0 * scale
    height = 1.0 * scale
    thickness = 0.05 * scale
    tilt_rad = np.radians(tilt_angle)
    
    x = np.array([0, width, width, 0, 0, width, width, 0]) + x_offset
    y = np.array([0, 0, 0, 0, height * np.cos(tilt_rad), height * np.cos(tilt_rad), 
                  height * np.cos(tilt_rad), height * np.cos(tilt_rad)]) + y_offset
    z = np.array([0, 0, thickness, thickness, 
                  height * np.sin(tilt_rad), height * np.sin(tilt_rad),
                  height * np.sin(tilt_rad) + thickness, height * np.sin(tilt_rad) + thickness]) + z_offset
    
    i = [0, 0, 0, 0, 4, 4, 0, 1, 1, 2, 2, 3]
    j = [1, 2, 4, 5, 5, 6, 3, 2, 5, 3, 6, 7]
    k = [2, 3, 5, 6, 6, 7, 4, 5, 6, 6, 7, 4]
    
    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=COLORS['solar'],
        opacity=0.9,
        name='太阳能板',
        hoverinfo='name',
        flatshading=True
    )


def create_solar_panel_array(rows=2, cols=3, spacing=2.5):
    """创建太阳能电池板阵列"""
    panels = []
    for i in range(rows):
        for j in range(cols):
            panel = create_solar_panel_3d(
                x_offset=j * spacing,
                y_offset=i * spacing * 0.8,
                z_offset=1.5,
                scale=1.0,
                tilt_angle=35
            )
            panels.append(panel)
    return panels


def create_battery_storage_3d(x_offset=0, y_offset=0, z_offset=0, soc=0.75):
    """
    创建3D储能电池模型
    """
    width = 1.5
    depth = 0.8
    height = 2.0
    
    x = np.array([0, width, width, 0, 0, width, width, 0]) + x_offset
    y = np.array([0, 0, depth, depth, 0, 0, depth, depth]) + y_offset
    z = np.array([0, 0, 0, 0, height, height, height, height]) + z_offset
    
    i = [0, 0, 0, 0, 4, 4, 0, 1, 1, 2, 2, 3]
    j = [1, 2, 4, 5, 5, 6, 3, 2, 5, 3, 6, 7]
    k = [2, 3, 5, 6, 6, 7, 4, 5, 6, 6, 7, 4]
    
    if soc > 0.6:
        color = '#00CED1'
    elif soc > 0.3:
        color = '#FFA500'
    else:
        color = '#FF4500'
    
    battery_case = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=0.85,
        name=f'储能电池 (SOC: {soc*100:.0f}%)',
        hoverinfo='name',
        flatshading=True
    )
    
    return [battery_case]


def create_battery_bank(count=4, soc_list=None):
    """创建储能电池组"""
    if soc_list is None:
        soc_list = [0.8, 0.65, 0.75, 0.9][:count]
    
    batteries = []
    spacing = 1.8
    
    for i in range(count):
        soc = soc_list[i] if i < len(soc_list) else 0.5
        battery_parts = create_battery_storage_3d(
            x_offset=10 + i * spacing,
            y_offset=0,
            z_offset=0,
            soc=soc
        )
        batteries.extend(battery_parts)
    
    return batteries


def create_load_building_3d(x_offset=0, y_offset=0, z_offset=0, 
                            building_type='residential', load_level=0.5):
    """
    创建3D负荷建筑模型
    """
    if building_type == 'residential':
        width, depth, height = 2.0, 1.5, 2.5
        color = '#4169E1'
    elif building_type == 'commercial':
        width, depth, height = 3.0, 2.0, 4.0
        color = '#32CD32'
    else:
        width, depth, height = 4.0, 3.0, 3.0
        color = '#708090'
    
    x = np.array([0, width, width, 0, 0, width, width, 0]) + x_offset
    y = np.array([0, 0, depth, depth, 0, 0, depth, depth]) + y_offset
    z = np.array([0, 0, 0, 0, height, height, height, height]) + z_offset
    
    i = [0, 0, 0, 0, 4, 4, 0, 1, 1, 2, 2, 3]
    j = [1, 2, 4, 5, 5, 6, 3, 2, 5, 3, 6, 7]
    k = [2, 3, 5, 6, 6, 7, 4, 5, 6, 6, 7, 4]
    
    building = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=0.8 + load_level * 0.2,
        name=f'{building_type.capitalize()} (负荷: {load_level*100:.0f}%)',
        hoverinfo='name',
        flatshading=True
    )
    
    return [building]


def create_load_district():
    """创建负荷区域"""
    buildings = []
    
    for i in range(3):
        load = 0.3 + random.random() * 0.5
        building = create_load_building_3d(
            x_offset=20 + i * 3,
            y_offset=0,
            z_offset=0,
            building_type='residential',
            load_level=load
        )
        buildings.extend(building)
    
    building = create_load_building_3d(
        x_offset=20,
        y_offset=4,
        z_offset=0,
        building_type='commercial',
        load_level=0.7
    )
    buildings.extend(building)
    
    building = create_load_building_3d(
        x_offset=25,
        y_offset=4,
        z_offset=0,
        building_type='industrial',
        load_level=0.85
    )
    buildings.extend(building)
    
    return buildings


def create_power_grid_tower(x_offset=0, y_offset=0):
    """创建电网塔架"""
    tower_height = 5.0
    
    x = [x_offset, x_offset, x_offset + 0.2, x_offset + 0.2,
         x_offset + 0.3, x_offset + 0.3, x_offset - 0.1, x_offset - 0.1]
    y = [y_offset, y_offset + 0.2, y_offset + 0.2, y_offset,
         y_offset + 0.1, y_offset + 0.1, y_offset + 0.1, y_offset + 0.1]
    z = [0, 0, 0, 0, tower_height, tower_height, tower_height, tower_height]
    
    i = [0, 0, 4, 4]
    j = [1, 2, 5, 6]
    k = [2, 3, 6, 7]
    
    tower = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color='#A0A0A0',
        opacity=0.9,
        name='电网接入点',
        hoverinfo='name'
    )
    
    wire = go.Scatter3d(
        x=[x_offset, x_offset + 5],
        y=[y_offset + 0.1, y_offset + 0.1],
        z=[tower_height - 0.5, tower_height - 1],
        mode='lines',
        line=dict(color='#FFD700', width=3),
        name='输电线',
        hoverinfo='name'
    )
    
    return [tower, wire]


def create_energy_flow_lines(solar_power, battery_power, grid_power, load_power):
    """创建能量流动线条"""
    flows = []
    
    if solar_power > 0:
        flow1 = go.Scatter3d(
            x=[4, 8, 12],
            y=[2, 2, 2],
            z=[2, 2, 2],
            mode='lines+markers',
            line=dict(color=COLORS['solar'], width=max(2, solar_power/10)),
            marker=dict(size=[0, 8, 0], color=COLORS['solar']),
            name=f'光伏出力: {solar_power:.1f} kW',
            hoverinfo='name'
        )
        flows.append(flow1)
    
    if battery_power != 0:
        if battery_power > 0:
            color = '#00FF00'
            name = f'电池放电: {battery_power:.1f} kW'
        else:
            color = '#FF6600'
            name = f'电池充电: {-battery_power:.1f} kW'
        
        flow2 = go.Scatter3d(
            x=[12, 12, 12],
            y=[0.5, 1.5, 2],
            z=[1, 1.5, 2],
            mode='lines+markers',
            line=dict(color=color, width=max(2, abs(battery_power)/10)),
            marker=dict(size=[0, 6, 0], color=color),
            name=name,
            hoverinfo='name'
        )
        flows.append(flow2)
    
    if grid_power != 0:
        if grid_power > 0:
            color = '#9370DB'
            name = f'购电: {grid_power:.1f} kW'
        else:
            color = '#00CED1'
            name = f'售电: {-grid_power:.1f} kW'
        
        flow3 = go.Scatter3d(
            x=[-3, 4, 12],
            y=[5, 3.5, 2],
            z=[4, 3, 2],
            mode='lines+markers',
            line=dict(color=color, width=max(2, abs(grid_power)/10)),
            marker=dict(size=[0, 6, 0], color=color),
            name=name,
            hoverinfo='name'
        )
        flows.append(flow3)
    
    if load_power > 0:
        flow4 = go.Scatter3d(
            x=[12, 16, 22],
            y=[2, 2, 2],
            z=[2, 2, 1.5],
            mode='lines+markers',
            line=dict(color=COLORS['load'], width=max(2, load_power/10)),
            marker=dict(size=[0, 8, 0], color=COLORS['load']),
            name=f'负荷消耗: {load_power:.1f} kW',
            hoverinfo='name'
        )
        flows.append(flow4)
    
    return flows


def create_ground_plane():
    """创建地面平面"""
    x = np.linspace(-5, 35, 10)
    y = np.linspace(-5, 15, 10)
    x, y = np.meshgrid(x, y)
    z = np.zeros_like(x)
    
    ground = go.Surface(
        x=x, y=y, z=z,
        colorscale=[[0, '#2d5016'], [1, '#3d6b1e']],
        showscale=False,
        hoverinfo='skip',
        opacity=0.7
    )
    return ground

print("✅ 3D模型函数定义完成!")

## 🔋 第4步: 定义微电网模拟器

In [ ]:
class MicrogridSimulator:
    """微电网数据模拟器"""
    
    def __init__(self, pv_capacity=100, battery_capacity=200, max_load=150):
        self.pv_capacity = pv_capacity
        self.battery_capacity = battery_capacity
        self.max_load = max_load
        self.battery_soc = 0.5
        self.time = datetime.now().replace(hour=6, minute=0, second=0)
        
        self.history = {
            'time': [],
            'pv_power': [],
            'battery_power': [],
            'load_power': [],
            'grid_power': [],
            'soc': []
        }
    
    def get_solar_irradiance(self, hour):
        """获取太阳辐照度"""
        if hour < 6 or hour > 19:
            return 0
        irradiance = np.sin((hour - 6) / 13 * np.pi)
        irradiance *= (0.8 + random.random() * 0.4)
        return max(0, irradiance)
    
    def get_load_profile(self, hour):
        """获取负荷曲线"""
        base_load = 0.3
        
        if 0 <= hour < 6:
            load = base_load + 0.1
        elif 6 <= hour < 9:
            load = base_load + 0.3 + (hour - 6) * 0.1
        elif 9 <= hour < 12:
            load = 0.7
        elif 12 <= hour < 14:
            load = 0.85
        elif 14 <= hour < 18:
            load = 0.65
        elif 18 <= hour < 21:
            load = 0.9
        else:
            load = 0.4
        
        load *= (0.9 + random.random() * 0.2)
        return min(1.0, load)
    
    def simulate_step(self):
        """模拟一个时间步"""
        hour = self.time.hour + self.time.minute / 60
        
        irradiance = self.get_solar_irradiance(hour)
        pv_power = self.pv_capacity * irradiance
        
        load_factor = self.get_load_profile(hour)
        load_power = self.max_load * load_factor
        
        power_balance = pv_power - load_power
        
        battery_power = 0
        
        if power_balance > 0:
            if self.battery_soc < 0.95:
                charge_power = min(power_balance, self.battery_capacity * 0.2)
                charge_energy = charge_power / 60
                self.battery_soc = min(0.95, self.battery_soc + charge_energy / self.battery_capacity)
                battery_power = -charge_power
                power_balance -= charge_power
        else:
            if self.battery_soc > 0.1:
                discharge_power = min(-power_balance, self.battery_capacity * 0.3)
                discharge_energy = discharge_power / 60
                self.battery_soc = max(0.1, self.battery_soc - discharge_energy / self.battery_capacity)
                battery_power = discharge_power
                power_balance += discharge_power
        
        grid_power = -power_balance
        
        self.history['time'].append(self.time)
        self.history['pv_power'].append(pv_power)
        self.history['battery_power'].append(battery_power)
        self.history['load_power'].append(load_power)
        self.history['grid_power'].append(grid_power)
        self.history['soc'].append(self.battery_soc)
        
        max_history = 1440
        for key in self.history:
            if len(self.history[key]) > max_history:
                self.history[key] = self.history[key][-max_history:]
        
        self.time += timedelta(minutes=1)
        
        return {
            'pv_power': pv_power,
            'battery_power': battery_power,
            'load_power': load_power,
            'grid_power': grid_power,
            'soc': self.battery_soc,
            'time': self.time
        }
    
    def get_battery_soc_list(self, count=4):
        base_soc = self.battery_soc
        return [base_soc + random.uniform(-0.1, 0.1) for _ in range(count)]

print("✅ 微电网模拟器定义完成!")

## 📊 第5步: 创建可视化函数

In [ ]:
def create_3d_system_view(simulator, state):
    """创建3D系统视图"""
    fig = go.Figure()
    
    # 添加地面
    fig.add_trace(create_ground_plane())
    
    # 添加光伏阵列
    for panel in create_solar_panel_array(rows=2, cols=3):
        fig.add_trace(panel)
    
    # 添加储能系统
    soc_list = simulator.get_battery_soc_list(4)
    for battery in create_battery_bank(count=4, soc_list=soc_list):
        fig.add_trace(battery)
    
    # 添加负荷区域
    for building in create_load_district():
        fig.add_trace(building)
    
    # 添加电网接入点
    for element in create_power_grid_tower(x_offset=-2, y_offset=5):
        fig.add_trace(element)
    
    # 添加能量流动线
    for flow in create_energy_flow_lines(
        solar_power=state['pv_power'],
        battery_power=state['battery_power'],
        grid_power=state['grid_power'],
        load_power=state['load_power']
    ):
        fig.add_trace(flow)
    
    # 添加标签
    labels = [
        go.Scatter3d(x=[3], y=[4], z=[4], mode='text',
                    text=['☀️ 光伏发电区'], textfont=dict(size=14, color='#FFD700'),
                    hoverinfo='skip', showlegend=False),
        go.Scatter3d(x=[13], y=[-1], z=[3], mode='text',
                    text=['🔋 储能系统'], textfont=dict(size=14, color='#00CED1'),
                    hoverinfo='skip', showlegend=False),
        go.Scatter3d(x=[25], y=[-1], z=[5], mode='text',
                    text=['🏠 负荷区域'], textfont=dict(size=14, color='#FF6347'),
                    hoverinfo='skip', showlegend=False),
        go.Scatter3d(x=[-2], y=[6], z=[6], mode='text',
                    text=['⚡ 电网'], textfont=dict(size=14, color='#9370DB'),
                    hoverinfo='skip', showlegend=False),
    ]
    for label in labels:
        fig.add_trace(label)
    
    # 设置布局
    fig.update_layout(
        title=dict(
            text=f'微电网数字孪生 - 3D系统视图 | 时间: {state["time"].strftime("%H:%M")}',
            font=dict(size=20, color='#00CED1')
        ),
        scene=dict(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
            zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
            bgcolor='#0a0a1a',
            camera=dict(
                eye=dict(x=1.5, y=-1.5, z=0.8),
                up=dict(x=0, y=0, z=1)
            ),
            aspectmode='data'
        ),
        paper_bgcolor='#1a1a2e',
        plot_bgcolor='#1a1a2e',
        height=600,
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=True,
        legend=dict(
            font=dict(color='#eee'),
            bgcolor='rgba(22, 33, 62, 0.8)',
            bordercolor='#00CED1',
            borderwidth=1
        )
    )
    
    return fig


def create_dashboard_view(simulator, state):
    """创建综合仪表盘视图"""
    history = simulator.history
    
    fig = make_subplots(
        rows=2, cols=3,
        specs=[
            [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
            [{"type": "scatter", "colspan": 2}, None, {"type": "pie"}]
        ],
        subplot_titles=('', '', '', '功率曲线', '', '能量构成'),
        row_heights=[0.35, 0.65],
        vertical_spacing=0.12
    )
    
    # 光伏仪表盘
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=state['pv_power'],
        title={'text': "☀️ 光伏出力 (kW)", 'font': {'color': '#FFD700', 'size': 14}},
        gauge={
            'axis': {'range': [0, simulator.pv_capacity], 'tickcolor': '#eee'},
            'bar': {'color': '#FFD700'},
            'bgcolor': '#16213e',
            'borderwidth': 2,
            'bordercolor': '#FFD700',
            'steps': [
                {'range': [0, simulator.pv_capacity*0.3], 'color': '#2d2d2d'},
                {'range': [simulator.pv_capacity*0.3, simulator.pv_capacity*0.7], 'color': '#3d3d3d'},
                {'range': [simulator.pv_capacity*0.7, simulator.pv_capacity], 'color': '#4d4d4d'}
            ]
        }
    ), row=1, col=1)
    
    # 储能SOC仪表盘
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=state['soc'] * 100,
        title={'text': "🔋 储能SOC (%)", 'font': {'color': '#00CED1', 'size': 14}},
        gauge={
            'axis': {'range': [0, 100], 'tickcolor': '#eee'},
            'bar': {'color': '#00CED1'},
            'bgcolor': '#16213e',
            'borderwidth': 2,
            'bordercolor': '#00CED1',
            'steps': [
                {'range': [0, 20], 'color': '#8B0000'},
                {'range': [20, 50], 'color': '#FFA500'},
                {'range': [50, 100], 'color': '#006400'}
            ]
        }
    ), row=1, col=2)
    
    # 负荷仪表盘
    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=state['load_power'],
        title={'text': "🏠 负荷消耗 (kW)", 'font': {'color': '#FF6347', 'size': 14}},
        gauge={
            'axis': {'range': [0, simulator.max_load], 'tickcolor': '#eee'},
            'bar': {'color': '#FF6347'},
            'bgcolor': '#16213e',
            'borderwidth': 2,
            'bordercolor': '#FF6347',
            'steps': [
                {'range': [0, simulator.max_load*0.5], 'color': '#2d2d2d'},
                {'range': [simulator.max_load*0.5, simulator.max_load*0.8], 'color': '#3d3d3d'},
                {'range': [simulator.max_load*0.8, simulator.max_load], 'color': '#4d4d4d'}
            ]
        }
    ), row=1, col=3)
    
    # 功率曲线
    times = list(range(len(history['time'])))
    
    fig.add_trace(go.Scatter(
        x=times, y=history['pv_power'],
        name='光伏', line=dict(color='#FFD700', width=2)
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=times, y=history['load_power'],
        name='负荷', line=dict(color='#FF6347', width=2)
    ), row=2, col=1)
    
    fig.add_trace(go.Scatter(
        x=times, y=[abs(p) for p in history['battery_power']],
        name='储能', line=dict(color='#00CED1', width=2)
    ), row=2, col=1)
    
    # 能量构成饼图
    pv_energy = max(0.1, state['pv_power'])
    battery_energy = max(0.1, abs(state['battery_power']))
    grid_energy = max(0.1, abs(state['grid_power']))
    
    fig.add_trace(go.Pie(
        labels=['光伏', '储能', '电网'],
        values=[pv_energy, battery_energy, grid_energy],
        hole=0.4,
        marker=dict(colors=['#FFD700', '#00CED1', '#9370DB']),
        textinfo='label+percent',
        textfont=dict(color='#eee')
    ), row=2, col=3)
    
    # 更新布局
    fig.update_layout(
        title=dict(
            text=f'微电网数字孪生仪表盘 - {state["time"].strftime("%Y-%m-%d %H:%M")}',
            font=dict(size=22, color='#00CED1'),
            x=0.5
        ),
        paper_bgcolor='#1a1a2e',
        plot_bgcolor='#16213e',
        font=dict(color='#eee'),
        height=650,
        showlegend=True,
        legend=dict(
            bgcolor='rgba(22, 33, 62, 0.8)',
            bordercolor='#00CED1',
            borderwidth=1
        )
    )
    
    fig.update_xaxes(gridcolor='#333', zerolinecolor='#444')
    fig.update_yaxes(gridcolor='#333', zerolinecolor='#444')
    
    return fig


def create_energy_flow_sankey(state):
    """创建能量流动桑基图"""
    labels = ['光伏', '储能', '电网', '负荷']
    
    flows_source = []
    flows_target = []
    flows_value = []
    flows_color = []
    
    if state['pv_power'] > 0:
        pv_to_load = min(state['pv_power'], state['load_power'])
        if pv_to_load > 0:
            flows_source.append(0)
            flows_target.append(3)
            flows_value.append(pv_to_load)
            flows_color.append('rgba(255, 215, 0, 0.6)')
        
        if state['battery_power'] < 0:
            flows_source.append(0)
            flows_target.append(1)
            flows_value.append(-state['battery_power'])
            flows_color.append('rgba(255, 215, 0, 0.6)')
        
        if state['grid_power'] < 0:
            flows_source.append(0)
            flows_target.append(2)
            flows_value.append(-state['grid_power'])
            flows_color.append('rgba(255, 215, 0, 0.6)')
    
    if state['battery_power'] > 0:
        flows_source.append(1)
        flows_target.append(3)
        flows_value.append(state['battery_power'])
        flows_color.append('rgba(0, 206, 209, 0.6)')
    
    if state['grid_power'] > 0:
        flows_source.append(2)
        flows_target.append(3)
        flows_value.append(state['grid_power'])
        flows_color.append('rgba(147, 112, 219, 0.6)')
    
    if not flows_value:
        flows_source = [0]
        flows_target = [3]
        flows_value = [0.1]
        flows_color = ['rgba(100, 100, 100, 0.3)']
    
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color='#1a1a2e', width=0.5),
            label=labels,
            color=['#FFD700', '#00CED1', '#9370DB', '#FF6347']
        ),
        link=dict(
            source=flows_source,
            target=flows_target,
            value=flows_value,
            color=flows_color
        )
    )])
    
    fig.update_layout(
        title=dict(
            text='微电网能量流动图',
            font=dict(size=20, color='#00CED1')
        ),
        paper_bgcolor='#1a1a2e',
        plot_bgcolor='#1a1a2e',
        font=dict(color='#eee', size=14),
        height=450
    )
    
    return fig

print("✅ 可视化函数定义完成!")

## 🎮 第6步: 创建交互式控制界面

In [ ]:
class MicrogridDigitalTwinUI:
    """微电网数字孪生交互式界面"""
    
    def __init__(self):
        self.simulator = MicrogridSimulator(
            pv_capacity=100,
            battery_capacity=200,
            max_load=150
        )
        self.current_state = None
        
        # 初始化模拟数据
        for _ in range(60):
            self.current_state = self.simulator.simulate_step()
        
        self._create_widgets()
    
    def _create_widgets(self):
        """创建控件"""
        # 标题
        self.title_html = widgets.HTML(
            value="""
            <div style='background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); 
                        padding: 20px; border-radius: 10px; margin-bottom: 15px; text-align: center;'>
                <h1 style='color: #00CED1; margin: 0;'>⚡ 微电网数字孪生系统 ⚡</h1>
                <p style='color: #aaa; margin: 10px 0 0 0;'>Microgrid Digital Twin - Interactive 3D Visualization</p>
            </div>
            """
        )
        
        # 控制滑块
        self.pv_slider = widgets.FloatSlider(
            value=100, min=10, max=500, step=10,
            description='光伏容量(kW):',
            style={'description_width': '110px'},
            layout=widgets.Layout(width='350px')
        )
        
        self.battery_slider = widgets.FloatSlider(
            value=200, min=50, max=1000, step=50,
            description='储能容量(kWh):',
            style={'description_width': '110px'},
            layout=widgets.Layout(width='350px')
        )
        
        self.load_slider = widgets.FloatSlider(
            value=150, min=50, max=500, step=10,
            description='最大负荷(kW):',
            style={'description_width': '110px'},
            layout=widgets.Layout(width='350px')
        )
        
        self.time_slider = widgets.IntSlider(
            value=12, min=0, max=23, step=1,
            description='模拟时间(时):',
            style={'description_width': '110px'},
            layout=widgets.Layout(width='350px')
        )
        
        # 视图选择
        self.view_dropdown = widgets.Dropdown(
            options=[
                ('🏗️ 3D系统视图', '3d'),
                ('📊 综合仪表盘', 'dashboard'),
                ('🔄 能量流动图', 'flow')
            ],
            value='dashboard',
            description='视图模式:',
            style={'description_width': '80px'},
            layout=widgets.Layout(width='200px')
        )
        
        # 更新按钮
        self.update_button = widgets.Button(
            description='🔄 更新视图',
            button_style='primary',
            layout=widgets.Layout(width='120px')
        )
        self.update_button.on_click(self._on_update)
        
        # 模拟按钮
        self.sim_button = widgets.Button(
            description='▶️ 模拟1小时',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )
        self.sim_button.on_click(self._on_simulate)
        
        # 状态显示
        self.status_html = widgets.HTML(value='')
        
        # 输出区域
        self.output = widgets.Output()
        
        # 布局
        controls1 = widgets.HBox([self.pv_slider, self.battery_slider])
        controls2 = widgets.HBox([self.load_slider, self.time_slider])
        buttons = widgets.HBox(
            [self.view_dropdown, self.update_button, self.sim_button],
            layout=widgets.Layout(justify_content='center', margin='10px 0')
        )
        
        self.ui = widgets.VBox([
            self.title_html,
            controls1,
            controls2,
            buttons,
            self.status_html,
            self.output
        ])
    
    def _on_update(self, button):
        """更新视图"""
        self._update_simulator()
        self._update_display()
    
    def _on_simulate(self, button):
        """模拟1小时"""
        self._update_simulator()
        for _ in range(60):
            self.current_state = self.simulator.simulate_step()
        self.time_slider.value = self.simulator.time.hour
        self._update_display()
    
    def _update_simulator(self):
        """更新模拟器参数"""
        self.simulator.pv_capacity = self.pv_slider.value
        self.simulator.battery_capacity = self.battery_slider.value
        self.simulator.max_load = self.load_slider.value
        
        hour = self.time_slider.value
        self.simulator.time = self.simulator.time.replace(hour=hour, minute=0)
        
        self.current_state = self.simulator.simulate_step()
    
    def _update_display(self):
        """更新显示"""
        with self.output:
            clear_output(wait=True)
            
            view = self.view_dropdown.value
            
            if view == '3d':
                fig = create_3d_system_view(self.simulator, self.current_state)
            elif view == 'flow':
                fig = create_energy_flow_sankey(self.current_state)
            else:
                fig = create_dashboard_view(self.simulator, self.current_state)
            
            fig.show()
        
        self._update_status()
    
    def _update_status(self):
        """更新状态显示"""
        s = self.current_state
        
        if s['grid_power'] > 0:
            grid_txt = f"<span style='color:#FF6347'>购电 {s['grid_power']:.1f} kW</span>"
        elif s['grid_power'] < 0:
            grid_txt = f"<span style='color:#00FF00'>售电 {-s['grid_power']:.1f} kW</span>"
        else:
            grid_txt = "<span style='color:#00CED1'>平衡</span>"
        
        if s['battery_power'] > 0:
            bat_txt = f"放电 {s['battery_power']:.1f} kW"
        elif s['battery_power'] < 0:
            bat_txt = f"充电 {-s['battery_power']:.1f} kW"
        else:
            bat_txt = "待机"
        
        self.status_html.value = f"""
        <div style='background:#16213e; padding:15px; border-radius:8px; 
                    display:flex; justify-content:space-around; flex-wrap:wrap; margin-top:10px;'>
            <div style='text-align:center; padding:8px;'>
                <div style='color:#FFD700; font-size:20px;'>☀️ {s['pv_power']:.1f} kW</div>
                <div style='color:#aaa; font-size:12px;'>光伏出力</div>
            </div>
            <div style='text-align:center; padding:8px;'>
                <div style='color:#00CED1; font-size:20px;'>🔋 {s['soc']*100:.1f}%</div>
                <div style='color:#aaa; font-size:12px;'>{bat_txt}</div>
            </div>
            <div style='text-align:center; padding:8px;'>
                <div style='color:#FF6347; font-size:20px;'>🏠 {s['load_power']:.1f} kW</div>
                <div style='color:#aaa; font-size:12px;'>负荷消耗</div>
            </div>
            <div style='text-align:center; padding:8px;'>
                <div style='font-size:20px;'>⚡ {grid_txt}</div>
                <div style='color:#aaa; font-size:12px;'>电网交互</div>
            </div>
            <div style='text-align:center; padding:8px;'>
                <div style='color:#fff; font-size:20px;'>🕐 {s['time'].strftime('%H:%M')}</div>
                <div style='color:#aaa; font-size:12px;'>当前时间</div>
            </div>
        </div>
        """
    
    def display(self):
        """显示界面"""
        display(self.ui)
        self._update_display()

print("✅ 交互式界面类定义完成!")

---

# 🚀 启动微电网数字孪生系统

运行下面的单元格启动完整的交互式系统！

In [ ]:
# 🎉 启动微电网数字孪生系统
print("🔄 正在初始化微电网数字孪生系统...")
print()

# 创建并显示系统
twin = MicrogridDigitalTwinUI()
twin.display()

print()
print("="*60)
print("✅ 系统已启动!")
print()
print("📖 使用说明:")
print("  • 拖动滑块调整光伏容量、储能容量、负荷和时间")
print("  • 选择不同的视图模式: 3D系统视图/综合仪表盘/能量流动图")
print("  • 点击'更新视图'按钮刷新显示")
print("  • 点击'模拟1小时'按钮运行一小时模拟")
print("  • 在3D视图中可以用鼠标旋转、缩放视角")
print("="*60)

---

## 🎯 单独查看各个视图

如果你想单独查看某个特定视图，可以运行下面的代码单元格。

In [ ]:
# 📊 单独显示3D系统视图
print("📊 生成3D系统视图...")

# 创建模拟器
sim = MicrogridSimulator(pv_capacity=100, battery_capacity=200, max_load=150)

# 设置时间为中午12点
sim.time = sim.time.replace(hour=12, minute=0)

# 运行模拟
for _ in range(60):
    state = sim.simulate_step()

# 创建3D视图
fig = create_3d_system_view(sim, state)
fig.show()

print(f"\n☀️ 光伏: {state['pv_power']:.1f} kW | 🔋 SOC: {state['soc']*100:.1f}% | 🏠 负荷: {state['load_power']:.1f} kW")

In [ ]:
# 📈 单独显示综合仪表盘
print("📈 生成综合仪表盘...")

# 使用之前的模拟器继续运行
for _ in range(60):
    state = sim.simulate_step()

# 创建仪表盘
fig = create_dashboard_view(sim, state)
fig.show()

In [ ]:
# 🔄 单独显示能量流动图
print("🔄 生成能量流动图...")

# 使用之前的模拟器
state = sim.simulate_step()

# 创建能量流动图
fig = create_energy_flow_sankey(state)
fig.show()

print(f"\n能量流动: 光伏 {state['pv_power']:.1f} kW → 负荷 {state['load_power']:.1f} kW")
if state['grid_power'] > 0:
    print(f"从电网购电: {state['grid_power']:.1f} kW")
elif state['grid_power'] < 0:
    print(f"向电网售电: {-state['grid_power']:.1f} kW")

---

## 📈 24小时模拟分析

运行24小时完整模拟，分析微电网全天运行情况。

In [ ]:
# 🕐 24小时模拟
print("🕐 运行24小时模拟...")

# 创建新的模拟器
sim_24h = MicrogridSimulator(pv_capacity=100, battery_capacity=200, max_load=150)
sim_24h.time = sim_24h.time.replace(hour=0, minute=0)

# 运行24小时
for _ in range(24 * 60):  # 24小时 * 60分钟
    state = sim_24h.simulate_step()

# 创建分析图表
history = sim_24h.history

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('功率曲线 (kW)', '电池SOC (%)', '电网交互功率 (kW)'),
    vertical_spacing=0.1
)

# 时间轴
hours = [t.hour + t.minute/60 for t in history['time']]

# 功率曲线
fig.add_trace(go.Scatter(x=hours, y=history['pv_power'], name='光伏', 
                         line=dict(color='#FFD700', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=hours, y=history['load_power'], name='负荷', 
                         line=dict(color='#FF6347', width=2)), row=1, col=1)

# SOC曲线
fig.add_trace(go.Scatter(x=hours, y=[s*100 for s in history['soc']], name='SOC',
                         line=dict(color='#00CED1', width=2), 
                         fill='tozeroy', fillcolor='rgba(0, 206, 209, 0.2)'), row=2, col=1)

# 电网交互
fig.add_trace(go.Scatter(x=hours, y=history['grid_power'], name='电网功率',
                         line=dict(color='#9370DB', width=2),
                         fill='tozeroy', fillcolor='rgba(147, 112, 219, 0.2)'), row=3, col=1)

# 添加零线
fig.add_hline(y=0, line_dash='dash', line_color='white', opacity=0.5, row=3, col=1)

fig.update_layout(
    title=dict(text='微电网24小时运行分析', font=dict(size=22, color='#00CED1'), x=0.5),
    paper_bgcolor='#1a1a2e',
    plot_bgcolor='#16213e',
    font=dict(color='#eee'),
    height=700,
    showlegend=True,
    legend=dict(bgcolor='rgba(22, 33, 62, 0.8)', bordercolor='#00CED1', borderwidth=1)
)

fig.update_xaxes(title_text='时间 (小时)', gridcolor='#333', range=[0, 24])
fig.update_yaxes(gridcolor='#333')

fig.show()

# 统计分析
print("\n" + "="*60)
print("📊 24小时运行统计")
print("="*60)
print(f"☀️ 光伏发电量: {sum(history['pv_power'])/60:.1f} kWh")
print(f"🏠 负荷用电量: {sum(history['load_power'])/60:.1f} kWh")
print(f"⚡ 从电网购电: {sum(p for p in history['grid_power'] if p > 0)/60:.1f} kWh")
print(f"💰 向电网售电: {-sum(p for p in history['grid_power'] if p < 0)/60:.1f} kWh")
print(f"🔋 电池SOC范围: {min(history['soc'])*100:.1f}% - {max(history['soc'])*100:.1f}%")
print(f"📈 自发自用率: {(1 - sum(p for p in history['grid_power'] if p > 0) / sum(history['load_power']))*100:.1f}%")

---

## 🎨 自定义场景

你可以修改下面的参数来创建自己的微电网场景。

In [ ]:
# ⚙️ 自定义微电网参数

# 修改这些参数来创建你自己的场景
PV_CAPACITY = 150      # 光伏装机容量 (kW)
BATTERY_CAPACITY = 300  # 储能容量 (kWh) 
MAX_LOAD = 120          # 最大负荷 (kW)
SIMULATION_HOUR = 14    # 模拟的时间点 (0-23小时)

print(f"🔧 创建自定义微电网场景:")
print(f"   ☀️ 光伏容量: {PV_CAPACITY} kW")
print(f"   🔋 储能容量: {BATTERY_CAPACITY} kWh")
print(f"   🏠 最大负荷: {MAX_LOAD} kW")
print(f"   🕐 模拟时间: {SIMULATION_HOUR}:00")
print()

# 创建自定义模拟器
custom_sim = MicrogridSimulator(
    pv_capacity=PV_CAPACITY,
    battery_capacity=BATTERY_CAPACITY,
    max_load=MAX_LOAD
)
custom_sim.time = custom_sim.time.replace(hour=SIMULATION_HOUR, minute=0)

# 运行模拟
for _ in range(30):
    state = custom_sim.simulate_step()

# 显示3D视图
fig = create_3d_system_view(custom_sim, state)
fig.show()

# 显示状态
print("\n" + "="*50)
print(f"☀️ 光伏出力: {state['pv_power']:.1f} kW ({state['pv_power']/PV_CAPACITY*100:.0f}%)")
print(f"🔋 电池SOC: {state['soc']*100:.1f}%")
print(f"🏠 负荷消耗: {state['load_power']:.1f} kW")
if state['grid_power'] > 0:
    print(f"⚡ 从电网购电: {state['grid_power']:.1f} kW")
elif state['grid_power'] < 0:
    print(f"💰 向电网售电: {-state['grid_power']:.1f} kW")
else:
    print(f"⚖️ 能量平衡: 自给自足")

---

## 📝 关于本系统

### 系统架构

```
┌─────────────────────────────────────────────────────────────┐
│                    微电网数字孪生系统                          │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ☀️ 光伏发电    ─────►  🔌 能量中心  ◄─────  ⚡ 电网        │
│                              │                              │
│                              │                              │
│   🔋 储能系统   ◄────────────┼────────────►  🏠 负荷        │
│                              │                              │
│                              ▼                              │
│                    📊 数据监控与可视化                        │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### 功能特性

- **3D可视化**: 光伏阵列、储能电池、负荷建筑的3D模型展示
- **实时模拟**: 基于时间的太阳辐照度和负荷曲线模拟
- **能量流动**: 可视化展示能量在各组件间的流动
- **交互控制**: 调整系统参数，实时观察运行变化
- **数据分析**: 功率曲线、SOC变化、能量统计

### 技术栈

- **Plotly**: 3D图形和交互式图表
- **ipywidgets**: 交互式控件
- **NumPy**: 数值计算

---

🎉 **感谢使用微电网数字孪生系统!** 🎉